## Tidying Up Data

* concepts
* unkown values
* NaN values
* filter/drop data
* find, filter and fix unknown values
* identify and remove duplicate data
* transform values using replace, map and apply

Tidy data is a term created in the paper: vita.had.co.nz/papers/tidy-data.pdf. Tidying of data is required for many reasons including:

* The names of the vars are different from what you require
* Missing data
* Values are not in the units that you require
* The period is not you want
* Categorical vs. quantitative
* Noise data
* Incorrect type
* Incorrect axes
* Wrong level
* Duplicated data

Characteristics of tidy data:

* Each var is in one column
* Each observation of the var is in a diff row
* One table for each kind of var
* If multiple tables, they should be relatable
* Quanlitative and categorical var have mappings to values useful for analysis

In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 10)

## Working with missing data

Data is **missing** when it has a value of NaN.

In [2]:
df = pd.DataFrame(np.arange(0, 15).reshape(5, 3),
                 index=['a', 'b', 'c', 'd', 'e'],
                 columns=['c1', 'c2', 'c3'])
df

   c1  c2  c3
a   0   1   2
b   3   4   5
c   6   7   8
d   9  10  11
e  12  13  14

In [3]:
# add NaN values
df['c4'] = np.nan
df.loc['f'] = np.arange(15, 19)
df.loc['g'] = np.nan
df['c5'] = np.nan
df['c4']['a'] = 20
df

   c1  c2  c3  c4  c5
a   0   1   2  20 NaN
b   3   4   5 NaN NaN
c   6   7   8 NaN NaN
d   9  10  11 NaN NaN
e  12  13  14 NaN NaN
f  15  16  17  18 NaN
g NaN NaN NaN NaN NaN

## Determining NaN values


In [4]:
df.isnull()

      c1     c2     c3     c4    c5
a  False  False  False  False  True
b  False  False  False   True  True
c  False  False  False   True  True
d  False  False  False   True  True
e  False  False  False   True  True
f  False  False  False  False  True
g   True   True   True   True  True

In [5]:
df.isnull().sum()

c1    1
c2    1
c3    1
c4    5
c5    7
dtype: int64

In [6]:
df.isnull().sum().sum()

15

In [9]:
# or
(len(df) - df.count()).sum()

15

In [10]:
df.notnull()

      c1     c2     c3     c4     c5
a   True   True   True   True  False
b   True   True   True  False  False
c   True   True   True  False  False
d   True   True   True  False  False
e   True   True   True  False  False
f   True   True   True   True  False
g  False  False  False  False  False

## Select out or dropping missing data


In [11]:
df.c4[df.c4.notnull()]

a    20
f    18
Name: c4, dtype: float64

In [12]:
# or
df.c4.dropna()

a    20
f    18
Name: c4, dtype: float64

In [13]:
# drop rows that contain at least one NaN value
df.dropna()

Empty DataFrame
Columns: [c1, c2, c3, c4, c5]
Index: []

In [14]:
df.dropna(how='all')

   c1  c2  c3  c4  c5
a   0   1   2  20 NaN
b   3   4   5 NaN NaN
c   6   7   8 NaN NaN
d   9  10  11 NaN NaN
e  12  13  14 NaN NaN
f  15  16  17  18 NaN

In [15]:
df.dropna(how='all', axis=1)

   c1  c2  c3  c4
a   0   1   2  20
b   3   4   5 NaN
c   6   7   8 NaN
d   9  10  11 NaN
e  12  13  14 NaN
f  15  16  17  18
g NaN NaN NaN NaN

In [16]:
df2 = df.copy()
df2.ix['g'].c1 = 0
df2.ix['g'].c3 = 0
df2

   c1  c2  c3  c4  c5
a   0   1   2  20 NaN
b   3   4   5 NaN NaN
c   6   7   8 NaN NaN
d   9  10  11 NaN NaN
e  12  13  14 NaN NaN
f  15  16  17  18 NaN
g   0 NaN   0 NaN NaN

In [17]:
df2.dropna(how='any', axis=1)

   c1  c3
a   0   2
b   3   5
c   6   8
d   9  11
e  12  14
f  15  17
g   0   0

In [18]:
# or use a threshold value
df.dropna(thresh=5, axis=1)

   c1  c2  c3
a   0   1   2
b   3   4   5
c   6   7   8
d   9  10  11
e  12  13  14
f  15  16  17
g NaN NaN NaN

## Handles NaN values in mathematical operations


In [19]:
a = np.array([1, 2, np.nan, 3])
s = pd.Series(a)
a.mean(), s.mean()

(nan, 2.0)

In [20]:
df.c4

a    20
b   NaN
c   NaN
d   NaN
e   NaN
f    18
g   NaN
Name: c4, dtype: float64

In [21]:
s = df.c4
s.sum()

38.0

In [22]:
s.mean()

19.0

In [23]:
s.cumsum()

a    20
b   NaN
c   NaN
d   NaN
e   NaN
f    38
g   NaN
Name: c4, dtype: float64

In [24]:
s+1

a    21
b   NaN
c   NaN
d   NaN
e   NaN
f    19
g   NaN
Name: c4, dtype: float64

## Filling in missing data


In [26]:
filled = df.fillna(0)
filled

   c1  c2  c3  c4  c5
a   0   1   2  20   0
b   3   4   5   0   0
c   6   7   8   0   0
d   9  10  11   0   0
e  12  13  14   0   0
f  15  16  17  18   0
g   0   0   0   0   0

In [27]:
df.mean()

c1     7.5
c2     8.5
c3     9.5
c4    19.0
c5     NaN
dtype: float64

In [28]:
filled.mean()

c1    6.428571
c2    7.285714
c3    8.142857
c4    5.428571
c5    0.000000
dtype: float64

In [29]:
# 
df.fillna(0, limit=2)

   c1  c2  c3  c4  c5
a   0   1   2  20   0
b   3   4   5   0   0
c   6   7   8   0 NaN
d   9  10  11 NaN NaN
e  12  13  14 NaN NaN
f  15  16  17  18 NaN
g   0   0   0 NaN NaN

## Forward and backward filling of mising data


In [30]:
# forward, last known value
df.c4.fillna(method='ffill')

a    20
b    20
c    20
d    20
e    20
f    18
g    18
Name: c4, dtype: float64

In [31]:
df.c4.fillna(method='bfill')

a    20
b    18
c    18
d    18
e    18
f    18
g   NaN
Name: c4, dtype: float64

Also we can use `pd.ffill` or `pd.bfill`.

## Filling using index labels

In [32]:
fill_values = pd.Series([100, 101, 102], index=['a', 'e', 'g'])
fill_values

a    100
e    101
g    102
dtype: int64

In [34]:
df.c4.fillna(fill_values)

a     20
b    NaN
c    NaN
d    NaN
e    101
f     18
g    102
Name: c4, dtype: float64

In [36]:
df.fillna(df.mean())

     c1    c2    c3  c4  c5
a   0.0   1.0   2.0  20 NaN
b   3.0   4.0   5.0  19 NaN
c   6.0   7.0   8.0  19 NaN
d   9.0  10.0  11.0  19 NaN
e  12.0  13.0  14.0  19 NaN
f  15.0  16.0  17.0  18 NaN
g   7.5   8.5   9.5  19 NaN

## Interpolation of missing values


In [37]:
s = pd.Series([1, np.nan, np.nan, np.nan, 2])
s.interpolate()

0    1.00
1    1.25
2    1.50
3    1.75
4    2.00
dtype: float64

In [38]:
# time series
import datetime
ts = pd.Series([1, np.nan, 2],
              index=[datetime.datetime(2004, 1, 1),
                     datetime.datetime(2004, 2, 1),
                     datetime.datetime(2004, 4, 1)])
ts

2004-01-01     1
2004-02-01   NaN
2004-04-01     2
dtype: float64

In [39]:
ts.interpolate()

2004-01-01    1.0
2004-02-01    1.5
2004-04-01    2.0
dtype: float64

In [40]:
ts.interpolate(method='time')

2004-01-01    1.000000
2004-02-01    1.340659
2004-04-01    2.000000
dtype: float64

In [41]:
# numeric index labels
s = pd.Series([0, np.nan, 100], index=[0, 1, 10])
s

0       0
1     NaN
10    100
dtype: float64

In [42]:
s.interpolate()

0       0
1      50
10    100
dtype: float64

In [43]:
s.interpolate(method='values')

0       0
1      10
10    100
dtype: float64

## Handling duplicate data


In [44]:
data = pd.DataFrame({'a': ['x'] * 3 + ['y'] * 4,
                    'b': [1, 1, 2, 3, 3, 4, 4]})
data

   a  b
0  x  1
1  x  1
2  x  2
3  y  3
4  y  3
5  y  4
6  y  4

In [45]:
data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

In [46]:
data.drop_duplicates()

   a  b
0  x  1
2  x  2
3  y  3
5  y  4

In [48]:
data.drop_duplicates(keep='last')

   a  b
1  x  1
2  x  2
4  y  3
6  y  4

In [49]:
# add a column
data['c'] = range(7)
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6    False
dtype: bool

In [50]:
data.drop_duplicates(['a', 'b'])

   a  b  c
0  x  1  0
2  x  2  2
3  y  3  3
5  y  4  5

## Transforming data

* Mapping
* Replacing
* Applying
* Removing

## Mapping

In [51]:
## mapping
x = pd.Series({'one': 1, 'two': 2, 'three': 3})
y = pd.Series({1: 'a', 2: 'b', 3: 'c'})
x

one      1
three    3
two      2
dtype: int64

In [52]:
y

1    a
2    b
3    c
dtype: object

In [53]:
x.map(y)

one      a
three    c
two      b
dtype: object

In [55]:
x = pd.Series({'one': 1, 'two': 2, 'three': 3})
y = pd.Series({1: 'a', 2: 'b'})
x.map(y)

one        a
three    NaN
two        b
dtype: object

## Replacing values


In [57]:
s = pd.Series([0., 1., 2., 3., 2., 4.])
s

0    0
1    1
2    2
3    3
4    2
5    4
dtype: float64

In [58]:
s.replace(2, 5)

0    0
1    1
2    5
3    3
4    5
5    4
dtype: float64

In [59]:
s.replace([0, 1, 2, 3, 4], [4, 3, 2, 1, 0])

0    4
1    3
2    2
3    1
4    2
5    0
dtype: float64

In [60]:
s.replace({0: 10, 1: 100})

0     10
1    100
2      2
3      3
4      2
5      4
dtype: float64

In [61]:
## Replacing values in DataFrame
df = pd.DataFrame({'a': [0, 1, 2, 3, 4], 'b': [5, 6, 7, 8, 9]})
df

   a  b
0  0  5
1  1  6
2  2  7
3  3  8
4  4  9

In [62]:
df.replace({'a': 1, 'b': 8}, 100)

     a    b
0    0    5
1  100    6
2    2    7
3    3  100
4    4    9

In [64]:
s[0] = 10
s

0    10
1     1
2     2
3     3
4     2
5     4
dtype: float64

In [65]:
s.replace([1, 2, 3], method='pad')

0    10
1    10
2    10
3    10
4    10
5     4
dtype: float64

## Applying functions to transform data


In [66]:
s = pd.Series(np.arange(0, 5))
s.apply(lambda v: v*2)

0    0
1    2
2    4
3    6
4    8
dtype: int64

In [67]:
df = pd.DataFrame(np.arange(12).reshape(4, 3), columns=['a', 'b', 'c'])
df

   a   b   c
0  0   1   2
1  3   4   5
2  6   7   8
3  9  10  11

In [68]:
df.apply(lambda col: col.sum())

a    18
b    22
c    26
dtype: int64

In [69]:
# rows
df.apply(lambda row: row.sum(), axis=1)

0     3
1    12
2    21
3    30
dtype: int64

In [71]:
df['interim'] = df.apply(lambda r: r.a * r.b, axis=1)
df

   a   b   c  interim
0  0   1   2        0
1  3   4   5       12
2  6   7   8       42
3  9  10  11       90

In [73]:
df['result'] = df.apply(lambda r: r.c + r.interim, axis=1)
df

   a   b   c  interim  result
0  0   1   2        0       2
1  3   4   5       12      17
2  6   7   8       42      50
3  9  10  11       90     101

In [74]:
df = pd.DataFrame(np.arange(15).reshape(3, 5))
df.loc[1, 2] = np.nan
df

    0   1   2   3   4
0   0   1   2   3   4
1   5   6 NaN   8   9
2  10  11  12  13  14

In [76]:
df.dropna().apply(lambda x: x.sum(), axis=1)

0    10
2    60
dtype: float64

In [77]:
df.applymap(lambda x: '%.2f' % x)

       0      1      2      3      4
0   0.00   1.00   2.00   3.00   4.00
1   5.00   6.00    nan   8.00   9.00
2  10.00  11.00  12.00  13.00  14.00